<a href="https://colab.research.google.com/github/jwyangyin/TFM/blob/main/Notebook_3_%E2%80%94_Sistema_de_Recomendaci%C3%B3n_H%C3%ADbrido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Sistema de Recomendación Híbrido</h1>

<h2>Combinación de Filtrado Colaborativo Item-to-Item y Recomendación Basada en Contenido</h2>

<p>
En este notebook se desarrolla un sistema de recomendación híbrido que combina dos enfoques
complementarios: el filtrado colaborativo <i>item-to-item</i> y la recomendación basada en
contenido. El objetivo es aprovechar las fortalezas de ambos métodos para generar
recomendaciones más robustas y equilibradas.
</p>

<h2>1. Introducción y motivación</h2>

<p>
Los sistemas de recomendación presentan diferentes ventajas y limitaciones en función del
enfoque utilizado. Mientras que el filtrado colaborativo es capaz de capturar patrones de
comportamiento colectivo entre usuarios, los sistemas basados en contenido permiten generar
recomendaciones incluso en ausencia de interacciones previas.
</p>

<p>
En este trabajo ya se han desarrollado y evaluado ambos enfoques de forma independiente.
En este tercer sistema se propone un modelo híbrido que combina la información procedente
de ambos sistemas con el fin de mejorar la calidad, estabilidad y cobertura de las
recomendaciones.
</p>

<p>
Este enfoque híbrido es ampliamente utilizado en entornos reales, ya que permite mitigar
problemas como el <i>cold start</i>, la dispersión de datos o la falta de información textual
o de interacción en determinados productos.
</p>

<h3>1.1 Objetivo del sistema híbrido</h3>

<p>
El objetivo principal de este sistema es generar recomendaciones de productos combinando:
</p>

<ul>
  <li>La similitud basada en comportamiento de usuarios (filtrado colaborativo item-to-item).</li>
  <li>La similitud semántica basada en descripciones textuales (content-based).</li>
</ul>

<p>
Para ello, se construye un score combinado que pondera ambos enfoques, permitiendo generar
rankings de recomendación más equilibrados y robustos que los obtenidos por cada sistema
de forma independiente.
</p>

Dado que estos enfoques han sido desarrollados y evaluados en apartados previos, en este sistema se presentan únicamente de forma resumida.

<h3>1.2 Estructura del notebook</h3>

<p>
El desarrollo del sistema híbrido se organiza en los siguientes apartados:
</p>

<ol>
  <li>Carga y exploración inicial de los datos.</li>
  <li>Preparación de los datos para ambos sistemas base.</li>
  <li>Revisión de los sistemas item-to-item y basado en contenido.</li>
  <li>Definición de la estrategia híbrida.</li>
  <li>Implementación del sistema de recomendación híbrido.</li>
  <li>Evaluación cualitativa y cuantitativa.</li>
  <li>Conclusiones del sistema híbrido.</li>
</ol>

In [ ]:
# Importamos las librerías necesarias:
# Librerías básicas
import numpy as np
import pandas as pd
import json

# Visualización
import matplotlib.pyplot as plt

# Texto y similitud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Matrices dispersas
from scipy.sparse import csr_matrix

# Utilidades
from collections import defaultdict


<h2>2. Carga de los datos</h2>

<p>
Se utilizan los mismos datasets de la categoría <i>All_Beauty</i> empleados en los sistemas
anteriores, correspondientes al conjunto <strong>Amazon Reviews 2023</strong>.
</p>

<p>
Se cargan tanto el fichero de reseñas (interacciones usuario-producto) como el fichero de
metadatos de productos, necesarios para la construcción de los sistemas colaborativo y
basado en contenido.
</p>

In [ ]:
# Montamos nuestro Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Definimos las rutas donde están los archivos JSONL subidos a Google Colab:
path_reviews = '/content/drive/My Drive/Colab Notebooks/All_Beauty.jsonl'
path_meta   = '/content/drive/My Drive/Colab Notebooks/meta_All_Beauty.jsonl'

# Cargamos el archivo de reseñas:
reviews = []
with open(path_reviews, "r") as f:
    for line in f:
        reviews.append(json.loads(line.strip()))

df_reviews = pd.DataFrame(reviews)
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [ ]:
# Cargamos el archivo de meta:
meta = []
with open(path_meta, "r") as f:
    for line in f:
        meta.append(json.loads(line.strip()))
df_meta = pd.DataFrame(meta)
df_meta.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,None


In [ ]:
print("Reviews:", df_reviews.shape)
print("Meta:", df_meta.shape)

Reviews: (701528, 10)
Meta: (112590, 14)


<h3>2.1 Inspección inicial de los datos</h3>

<p>
A continuación se realiza una inspección básica de los datasets para verificar su estructura,
tipos de datos y variables disponibles.
</p>

In [ ]:
df_reviews.head(3), df_meta.head(3)

(   rating                                      title  \
 0     5.0  Such a lovely scent but not overpowering.   
 1     4.0     Works great but smells a little weird.   
 2     5.0                                       Yes!   
 
                                                 text images        asin  \
 0  This spray is really nice. It smells really go...     []  B00YQ6X8EO   
 1  This product does what I need it to do, I just...     []  B081TJ8YS3   
 2                          Smells good, feels great!     []  B07PNNCSP9   
 
   parent_asin                       user_id      timestamp  helpful_vote  \
 0  B00YQ6X8EO  AGKHLEW2SOWHNMFQIJGBECAF7INQ  1588687728923             0   
 1  B081TJ8YS3  AGKHLEW2SOWHNMFQIJGBECAF7INQ  1588615855070             1   
 2  B097R46CSY  AE74DYR3QUGVPZJ3P7RFWBGIX7XQ  1589665266052             2   
 
    verified_purchase  
 0               True  
 1               True  
 2               True  ,
   main_category                                         

<h2>3. Análisis exploratorio y preparación de datos</h2>

<p>
En este apartado se realiza un análisis exploratorio resumido de los datos y se preparan
los conjuntos necesarios para la construcción del sistema híbrido. Dado que los datasets
ya han sido analizados en los sistemas anteriores, el EDA se centra en los aspectos
relevantes para la combinación de ambos enfoques.
</p>

<h3>3.1 Selección de variables relevantes</h3>

<p>
Para el sistema híbrido se requieren dos tipos de información:
</p>

<ul>
  <li><strong>Interacciones usuario-producto</strong> (para el filtrado colaborativo).</li>
  <li><strong>Información textual de los productos</strong> (para el sistema basado en contenido).</li>
</ul>

<p>
Por ello, se seleccionan únicamente las columnas necesarias de cada dataset, simplificando
el procesamiento y reduciendo el consumo de memoria.
</p>

In [ ]:
# Columnas relevantes de reviews
cols_reviews = ["user_id", "parent_asin", "rating", "timestamp"]
df_reviews_sel = df_reviews[cols_reviews].copy()

# Columnas relevantes de meta
cols_meta = ["parent_asin", "title", "description", "features"]
df_meta_sel = df_meta[cols_meta].copy()

df_reviews_sel.shape, df_meta_sel.shape

((701528, 4), (112590, 4))

<h3>3.2 Limpieza básica de los datos</h3>

<p>
Se realiza una limpieza básica de los datos, que incluye:
</p>

<ul>
  <li>Eliminación de registros con identificadores faltantes.</li>
  <li>Conversión de tipos de datos cuando es necesario.</li>
  <li>Normalización de campos textuales para su posterior procesamiento.</li>
</ul>

<p>
Este paso es fundamental para garantizar la consistencia del sistema híbrido, ya que
errores en los identificadores afectarían tanto al sistema colaborativo como al basado
en contenido.
</p>

In [ ]:
# Eliminamos registros sin identificadores clave
df_reviews_sel = df_reviews_sel.dropna(subset=["user_id", "parent_asin"])
df_meta_sel = df_meta_sel.dropna(subset=["parent_asin"])

# Aseguramos tipos de datos
df_reviews_sel["user_id"] = df_reviews_sel["user_id"].astype(str)
df_reviews_sel["parent_asin"] = df_reviews_sel["parent_asin"].astype(str)
df_meta_sel["parent_asin"] = df_meta_sel["parent_asin"].astype(str)

df_reviews_sel.shape, df_meta_sel.shape

((701528, 4), (112590, 4))

<h3>3.3 Análisis exploratorio resumido</h3>

<p>
Antes de la preparación final de los datos, se realiza un análisis exploratorio breve
para confirmar la naturaleza dispersa del dataset y justificar los filtros de actividad
mínima empleados posteriormente.
</p>

In [ ]:
n_users = df_reviews_sel["user_id"].nunique()
n_items = df_reviews_sel["parent_asin"].nunique()
n_interactions = len(df_reviews_sel)

n_users, n_items, n_interactions

(631986, 112565, 701528)

<p>
Los resultados confirman que el dataset presenta un número elevado de productos y usuarios,
con una cantidad relativamente baja de interacciones por entidad. Esta característica es
típica de los sistemas de recomendación reales y refuerza la necesidad de aplicar técnicas
de filtrado y combinación de enfoques para mejorar la estabilidad del modelo.
</p>

<h3>3.4 Filtrado por actividad mínima</h3>

<p>
Para reducir la dispersión extrema y mejorar la calidad de las similitudes calculadas,
se aplica un filtrado de actividad mínima tanto a usuarios como a productos.
</p>

<ul>
  <li>Se eliminan usuarios con menos de 3 interacciones.</li>
  <li>Se eliminan productos con menos de 3 interacciones.</li>
</ul>

<p>
Este criterio ya ha sido validado en los sistemas anteriores y permite obtener un subconjunto
de datos más estable sin perder representatividad.
</p>

In [ ]:
# Conteo de interacciones
user_counts = df_reviews_sel["user_id"].value_counts()
item_counts = df_reviews_sel["parent_asin"].value_counts()

# Usuarios y productos válidos
valid_users = user_counts[user_counts >= 3].index
valid_items = item_counts[item_counts >= 3].index

# Filtrado
df_reviews_filt = df_reviews_sel[
    df_reviews_sel["user_id"].isin(valid_users) &
    df_reviews_sel["parent_asin"].isin(valid_items)
].copy()

df_reviews_filt.shape

(33305, 4)

<h3>3.5 Resultado tras el filtrado</h3>

<p>
Tras aplicar el filtrado de actividad mínima, se obtiene un subconjunto de datos más
equilibrado, que será utilizado para construir los componentes colaborativo y basado
en contenido del sistema híbrido.
</p>

<p>
Este conjunto conserva la diversidad del catálogo y reduce el ruido introducido por
interacciones extremadamente escasas.
</p>

In [ ]:
df_reviews_filt["user_id"].nunique(), df_reviews_filt["parent_asin"].nunique(), len(df_reviews_filt)

(9016, 13671, 33305)

<h2>4. Construcción de los sistemas base</h2>

<p>
En este apartado se construyen los dos sistemas de recomendación que servirán como base
para el modelo híbrido:
</p>

<ul>
  <li>Un sistema de filtrado colaborativo <i>item-to-item</i>, basado en interacciones usuario-producto.</li>
  <li>Un sistema de recomendación basado en contenido, utilizando información textual de los productos.</li>
</ul>

<p>
Ambos sistemas ya han sido desarrollados y evaluados en notebooks anteriores. En este
caso, se reconstruyen de forma simplificada y modular con el objetivo de combinarlos
posteriormente en un único modelo híbrido.
</p>

<h3>4.1 Sistema base 1: Filtrado colaborativo item-to-item</h3>

<p>
El sistema de filtrado colaborativo se basa en la hipótesis de que productos que han sido
valorados de forma similar por los mismos usuarios tienden a ser similares entre sí.
</p>

<p>
Para implementar este enfoque se construye una matriz usuario–producto a partir de las
interacciones disponibles, y se calcula la similitud entre productos utilizando la
similitud del coseno.
</p>

In [ ]:
# Índices
user_ids = df_reviews_filt["user_id"].unique()
item_ids = df_reviews_filt["parent_asin"].unique()

user_to_idx = {u: i for i, u in enumerate(user_ids)}
item_to_idx = {i: j for j, i in enumerate(item_ids)}

# Construcción de la matriz dispersa
rows = df_reviews_filt["user_id"].map(user_to_idx)
cols = df_reviews_filt["parent_asin"].map(item_to_idx)
data = df_reviews_filt["rating"].astype(float)

R = csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(item_ids)))

R.shape

(9016, 13671)

<p>
A partir de la matriz usuario–producto se calcula la similitud entre productos utilizando
la similitud del coseno. Esta métrica permite comparar patrones de valoración independientemente
de la escala absoluta de los ratings.
</p>

In [ ]:
# Similitud item-to-item (colaborativa)
item_similarity_cf = cosine_similarity(R.T)
item_similarity_cf.shape

(13671, 13671)

<h3>4.2 Sistema base 2: Recomendación basada en contenido</h3>

<p>
El sistema basado en contenido utiliza información textual asociada a cada producto para
medir similitud semántica entre ítems. A diferencia del filtrado colaborativo, este enfoque
no depende de interacciones previas de los usuarios.
</p>

<p>
En este caso se emplea una representación TF-IDF de los textos descriptivos y la similitud
del coseno para comparar productos en el espacio vectorial resultante.
</p>

In [ ]:
# Combinamos campos textuales relevantes
def join_text_fields(row):
    parts = []
    for col in ["title", "description", "features"]:
        val = row.get(col, "")
        if isinstance(val, list):
            parts.extend([str(v) for v in val if v])
        elif isinstance(val, str):
            parts.append(val)
    return " ".join(parts)

df_meta_sel["item_text"] = df_meta_sel.apply(join_text_fields, axis=1)

<p>
Se eliminan productos sin información textual útil y se conservan únicamente aquellos
presentes en el conjunto filtrado de interacciones, garantizando coherencia entre ambos
sistemas base.
</p>

In [ ]:
items_df = (
    df_meta_sel[["parent_asin", "item_text"]]
    .dropna()
    .drop_duplicates("parent_asin")
)

# Nos quedamos solo con productos presentes en el filtrado colaborativo
items_df = items_df[items_df["parent_asin"].isin(item_ids)].reset_index(drop=True)

items_df.shape

(13671, 2)

<p>
El texto de cada producto se transforma en una representación vectorial mediante TF-IDF.
Se limita el vocabulario y se eliminan términos poco informativos para mejorar la
eficiencia y estabilidad del modelo.
</p>

In [ ]:
tfidf = TfidfVectorizer(
    max_features=10000,
    stop_words="english"
)

tfidf_matrix = tfidf.fit_transform(items_df["item_text"])

item_similarity_content = cosine_similarity(tfidf_matrix)

item_similarity_content.shape

(13671, 13671)

<h3>4.3 Resumen de los sistemas base</h3>

<p>
En este apartado se han construido dos sistemas de recomendación complementarios:
</p>

<ul>
  <li>Un sistema colaborativo que captura patrones de comportamiento de usuarios.</li>
  <li>Un sistema basado en contenido que captura similitud semántica entre productos.</li>
</ul>

<p>
En el siguiente apartado se combinarán ambos sistemas mediante una estrategia híbrida
para generar recomendaciones más robustas.
</p>

<h2>5. Sistema de recomendación híbrido</h2>

<p>
En este apartado se define e implementa el sistema de recomendación híbrido. La idea principal
es combinar dos fuentes de evidencia:
</p>

<ul>
  <li><strong>Similitud colaborativa</strong> (item-to-item), basada en patrones de valoración de usuarios.</li>
  <li><strong>Similitud basada en contenido</strong>, basada en similitud semántica de los textos de producto.</li>
</ul>

<p>
El sistema híbrido combina ambas puntuaciones mediante una ponderación controlada por un parámetro
<code>&alpha;</code>, obteniendo una recomendación final más robusta.
</p>

<h3>5.1 Estrategia de combinación</h3>

<p>
Para un producto consulta, cada sistema base produce una lista de candidatos con una puntuación:
</p>

<ul>
  <li><code>score_cf(i)</code>: similitud colaborativa del producto candidato con el producto consulta.</li>
  <li><code>score_cb(i)</code>: similitud basada en contenido del producto candidato con el producto consulta.</li>
</ul>

<p>
La puntuación final se obtiene mediante una combinación lineal:
</p>

<p style="text-align:center; font-size: 18px;">
  <b>score_híbrido(i) = &alpha; · score_cb(i) + (1 − &alpha;) · score_cf(i)</b>
</p>

<p>
En este notebook se utiliza un valor fijo <code>&alpha; = 0.5</code> como configuración base (Nivel A),
lo cual asigna el mismo peso a ambos enfoques.
</p>

In [ ]:
alpha = 0.5  # Peso del componente basado en contenido (0.5 = balanceado)

<h3>5.2 Recuperación eficiente de candidatos (Top-N)</h3>

<p>
Para mejorar la escalabilidad, en lugar de calcular matrices completas de similitud entre todos los productos
(N×N), se emplea una estrategia <em>Top-N</em> basada en vecinos más cercanos:
</p>

<ul>
  <li>Para el sistema colaborativo: vecinos más cercanos sobre la matriz item-usuario (<code>R.T</code>).</li>
  <li>Para el sistema de contenido: vecinos más cercanos sobre la matriz TF-IDF.</li>
</ul>

<p>
Este enfoque reduce drásticamente el coste computacional y permite generar recomendaciones de forma eficiente.
</p>

In [ ]:
from sklearn.neighbors import NearestNeighbors

# k para candidatos (más alto que top_n final para poder fusionar)
k_candidates = 50

# 1) kNN colaborativo sobre R.T (items x users)
knn_cf = NearestNeighbors(metric="cosine", algorithm="brute")
knn_cf.fit(R.T)

# 2) kNN contenido sobre TF-IDF (items x terms)
knn_cb = NearestNeighbors(metric="cosine", algorithm="brute")
knn_cb.fit(tfidf_matrix)

print("kNN CF y kNN Content entrenados.")

kNN CF y kNN Content entrenados.


<h3>5.3 Mapeos coherentes entre sistemas</h3>

<p>
Para poder combinar ambas fuentes, es necesario disponer de mapeos consistentes entre:
</p>

<ul>
  <li><code>parent_asin</code> → índice interno (CF)</li>
  <li><code>parent_asin</code> → índice interno (Content)</li>
</ul>

<p>
En caso de que un producto exista en un sistema pero no en el otro, el híbrido manejará la ausencia
asignando puntuación 0 al componente no disponible.
</p>

In [ ]:
# CF mapeos
idx_to_item_cf = {j: asin for asin, j in item_to_idx.items()}

# Content mapeos (items_df tiene solo productos con texto y presentes en item_ids)
asin_to_idx_cb = pd.Series(items_df.index, index=items_df["parent_asin"]).to_dict()
idx_to_asin_cb = pd.Series(items_df["parent_asin"].values, index=items_df.index).to_dict()

<h3>5.4 Recomendadores base (CF y Content)</h3>

<p>
Se definen dos funciones auxiliares que, dado un producto consulta, devuelven candidatos y su score:
</p>

<ul>
  <li><strong>CF item-to-item</strong>: vecinos más cercanos según patrones de usuarios.</li>
  <li><strong>Content-based</strong>: vecinos más cercanos según similitud TF-IDF.</li>
</ul>

In [ ]:
def recommend_cf(query_asin, top_k=10):
    if query_asin not in item_to_idx:
        return []
    q_idx = item_to_idx[query_asin]
    distances, indices = knn_cf.kneighbors(R.T[q_idx], n_neighbors=top_k + 1)
    indices = indices.flatten()
    distances = distances.flatten()
    recs = []
    for idx, dist in zip(indices, distances):
        asin = idx_to_item_cf[idx]
        if asin == query_asin:
            continue
        score = 1 - dist  # cosine distance -> cosine similarity
        recs.append((asin, float(score)))
    return recs[:top_k]

def recommend_cb(query_asin, top_k=10):
    if query_asin not in asin_to_idx_cb:
        return []
    q_idx = asin_to_idx_cb[query_asin]
    distances, indices = knn_cb.kneighbors(tfidf_matrix[q_idx], n_neighbors=top_k + 1)
    indices = indices.flatten()
    distances = distances.flatten()
    recs = []
    for idx, dist in zip(indices, distances):
        asin = idx_to_asin_cb[idx]
        if asin == query_asin:
            continue
        score = 1 - dist
        recs.append((asin, float(score)))
    return recs[:top_k]

<h3>5.5 Recomendador híbrido</h3>

<p>
El recomendador híbrido fusiona las listas candidatas de ambos sistemas y calcula un score final.
Se emplea una unión de candidatos y, para cada producto, se combina:
</p>

<ul>
  <li><code>score_cb</code> (si existe; si no, 0)</li>
  <li><code>score_cf</code> (si existe; si no, 0)</li>
</ul>

<p>
Finalmente se devuelve el ranking Top-N según <code>score_híbrido</code>.
</p>

In [ ]:
def recommend_hybrid(query_asin, top_n=10, k_candidates=50, alpha=0.5):
    # Candidatos de cada sistema (más que top_n para fusionar)
    recs_cf = recommend_cf(query_asin, top_k=k_candidates)
    recs_cb = recommend_cb(query_asin, top_k=k_candidates)

    dict_cf = {asin: score for asin, score in recs_cf}
    dict_cb = {asin: score for asin, score in recs_cb}

    # Unión de candidatos
    all_asins = set(dict_cf.keys()).union(dict_cb.keys())

    rows = []
    for asin in all_asins:
        s_cf = dict_cf.get(asin, 0.0)
        s_cb = dict_cb.get(asin, 0.0)
        s_h = alpha * s_cb + (1 - alpha) * s_cf
        rows.append((asin, s_h, s_cb, s_cf))

    out = pd.DataFrame(rows, columns=["asin", "score_hybrid", "score_content", "score_cf"])
    out = out.sort_values("score_hybrid", ascending=False).head(top_n).reset_index(drop=True)

    return out

<h3>5.6 Ejemplo de uso del sistema híbrido</h3>

<p>
A continuación se muestra un ejemplo reproducible seleccionando un producto consulta y generando:
</p>

<ul>
  <li>Recomendaciones colaborativas (CF)</li>
  <li>Recomendaciones basadas en contenido (CB)</li>
  <li>Recomendaciones híbridas (HYB)</li>
</ul>

<p>
Esto permite comparar de forma directa el efecto de la combinación.
</p>

In [ ]:
rng = np.random.default_rng(7)
query_asin = item_ids[rng.integers(0, len(item_ids))]
query_asin

'B019AINPZ4'

In [ ]:
print("CF:", recommend_cf(query_asin, top_k=5))
print("CB:", recommend_cb(query_asin, top_k=5))
recommend_hybrid(query_asin, top_n=10, k_candidates=50, alpha=alpha)

CF: [('B001B68ZHW', 1.0), ('B079RFBFSS', 0.7808688094430304), ('B01JD1DJZE', 0.49029033784546017), ('B0737QRNQ7', 0.0), ('B07FYXKPKF', 0.0)]
CB: [('B08NJ5BTWG', 0.5166364229784092), ('B01CS3OWBY', 0.46965029457209473), ('B00K58HQRM', 0.45408753234467936), ('B01699WMHO', 0.4185580641659291), ('B07F7MBMJ2', 0.3934711882633426)]


,asin,score_hybrid,score_content,score_cf
0,B001B68ZHW,0.500000,0.000000,1.000000
1,B079RFBFSS,0.390434,0.000000,0.780869
2,B08NJ5BTWG,0.258318,0.516636,0.000000
3,B01JD1DJZE,0.245145,0.000000,0.490290
4,B01CS3OWBY,0.234825,0.469650,0.000000
5,B00K58HQRM,0.227044,0.454088,0.000000
6,B01699WMHO,0.209279,0.418558,0.000000
7,B07F7MBMJ2,0.196736,0.393471,0.000000
8,B07W397QG4,0.173601,0.347203,0.000000
9,B00HKI013Q,0.172137,0.344273,0.000000


<h2>6. Evaluación cualitativa y cuantitativa del sistema híbrido</h2>

<p>
En este apartado se evalúa el sistema de recomendación híbrido desarrollado en los apartados anteriores,
el cual combina información procedente del filtrado colaborativo <i>item-to-item</i> y del modelo
basado en contenido.
</p>

<p>
Dado que no se dispone de un <i>ground truth</i> explícito de preferencias de usuario (ratings, clics o compras),
la evaluación se realiza mediante una combinación de análisis cualitativo y cuantitativo,
enfocados a verificar la coherencia, estabilidad y utilidad potencial de las recomendaciones generadas.
</p>

<h3>6.1 Enfoque de evaluación</h3>

<p>
En sistemas híbridos de recomendación, la evaluación depende en gran medida de la disponibilidad de datos
de interacción. A diferencia de escenarios puramente colaborativos, en este trabajo no se dispone
de feedback explícito que permita calcular métricas clásicas como RMSE, Precision@K o Recall@K.
</p>

<p>
Por este motivo, la evaluación del sistema híbrido se apoya en dos pilares complementarios:
</p>

<ul>
  <li>
    <strong>Evaluación cualitativa</strong>: inspección manual de títulos y fragmentos textuales para
    verificar la coherencia semántica entre el producto consulta y los productos recomendados.
  </li>
  <li>
    <strong>Evaluación cuantitativa</strong>: análisis estadístico de los scores de similitud generados
    por el modelo híbrido en el conjunto Top-N, comprobando su distribución y comportamiento decreciente.
  </li>
</ul>

<p>
Este enfoque es habitual en sistemas híbridos cuando no existe un conjunto de test etiquetado,
y se considera adecuado como validación inicial del modelo.
</p>

<h3>6.2 Evaluación cualitativa del sistema híbrido</h3>

<p>
La evaluación cualitativa consiste en analizar manualmente las recomendaciones generadas por el sistema híbrido
para productos de ejemplo, observando si los productos recomendados:
</p>

<ul>
  <li>Pertenecen a categorías similares o relacionadas.</li>
  <li>Comparten características semánticas relevantes en sus descripciones.</li>
  <li>Reflejan tanto similitud de contenido como patrones derivados del comportamiento de usuarios.</li>
</ul>

<p>
Esta inspección permite verificar que la combinación de señales colaborativas y de contenido
produce recomendaciones más robustas que cada sistema por separado.
</p>

In [ ]:
# Ejemplo reproducible para evaluación cualitativa del sistema híbrido
rng = np.random.default_rng(42)
sample_asin = items_df["parent_asin"].iloc[rng.integers(0, len(items_df))]

recs_df = recommend_hybrid(sample_asin, top_n=10)
recs_df

,asin,score_hybrid,score_content,score_cf
0,B071F78NNF,0.454811,0.561795,0.347827
1,B00GMP6QBO,0.390434,0.000000,0.780869
2,B01D1SVS7G,0.276079,0.000000,0.552158
3,B08VKT39YX,0.155582,0.311164,0.000000
4,B075RQ54DZ,0.151678,0.303355,0.000000
5,B08VKNT86J,0.145352,0.290704,0.000000
6,B07DXCVK8G,0.141259,0.282518,0.000000
7,B07PG6XMWT,0.140853,0.281706,0.000000
8,B07PJH5N6G,0.140853,0.281706,0.000000
9,B098JX1V49,0.129725,0.259451,0.000000


<p>
A partir de los resultados obtenidos, se observa que los productos recomendados presentan
una clara coherencia semántica con el producto consulta, al tiempo que incorporan información
procedente del filtrado colaborativo, lo que refuerza la relevancia de las recomendaciones.
</p>

In [ ]:
# Análisis estadístico de los scores del sistema híbrido
scores = recs_df["score_hybrid"].astype(float).values
pd.Series(scores).describe()

,0
count,10.000000
mean,0.212663
std,0.119284
min,0.129725
25%,0.140954
50%,0.148515
75%,0.245955
max,0.454811


<p>
Los resultados muestran un descenso progresivo de los valores de similitud híbrida,
con una distribución estable y sin saltos abruptos entre puntuaciones consecutivas.
</p>

<p>
Esto sugiere que el sistema híbrido discrimina adecuadamente entre productos más y menos similares,
integrando de forma equilibrada la información colaborativa y la semántica.
</p>

<h3>6.4 Limitaciones de la evaluación</h3>

<p>
La evaluación realizada presenta ciertas limitaciones inherentes al enfoque híbrido adoptado:
</p>

<ul>
  <li>No se evalúa la satisfacción real de usuarios finales.</li>
  <li>No se dispone de un <i>ground truth</i> explícito para métricas clásicas de ranking.</li>
  <li>La calidad del sistema depende de la disponibilidad tanto de texto como de interacciones.</li>
</ul>

<p>
No obstante, estas limitaciones son coherentes con el alcance del trabajo y no invalidan
las conclusiones obtenidas.
</p>

<h3>6.5 Conclusión de la evaluación</h3>

<p>
En conjunto, la evaluación cualitativa y cuantitativa realizada indica que el sistema de
recomendación híbrido:
</p>

<ul>
  <li>Genera recomendaciones coherentes desde el punto de vista semántico.</li>
  <li>Presenta un comportamiento estable en los scores de similitud.</li>
  <li>Mejora la robustez frente a sistemas puramente colaborativos o basados en contenido.</li>
</ul>

<p>
Estos resultados validan el enfoque híbrido propuesto como una solución sólida y adecuada
para el contexto del problema abordado.
</p>


<h2>7. Conclusiones y líneas futuras</h2>

<p>
En este apartado se presentan las conclusiones generales del sistema de recomendación híbrido
desarrollado, así como posibles líneas de mejora y extensión para trabajos futuros.
</p>

<h3>7.1 Conclusiones del sistema híbrido</h3>

<p>
El objetivo principal de este trabajo ha sido diseñar, implementar y evaluar un sistema de
recomendación híbrido que combine señales procedentes del filtrado colaborativo <i>item-to-item</i>
y de un modelo basado en contenido.
</p>

<p>
A partir de los resultados obtenidos, se pueden extraer las siguientes conclusiones:
</p>

<ul>
  <li>
    El sistema híbrido es capaz de generar recomendaciones semánticamente coherentes,
    integrando información textual de los productos y patrones de comportamiento derivados
    de las interacciones de usuarios.
  </li>
  <li>
    La combinación lineal de ambos enfoques permite compensar las limitaciones individuales
    de los sistemas base, mejorando la robustez del recomendador.
  </li>
  <li>
    Los scores de similitud presentan un comportamiento estable y ordenado, lo que indica
    que el modelo discrimina adecuadamente entre productos más y menos relevantes.
  </li>
</ul>

<p>
En conjunto, el sistema híbrido propuesto constituye una solución sólida y adecuada como
línea base para escenarios reales donde los datos son dispersos o incompletos.
</p>

<h3>7.2 Comparación con los sistemas base</h3>

<p>
La comparación con los sistemas desarrollados en notebooks anteriores permite observar que:
</p>

<ul>
  <li>
    El sistema de filtrado colaborativo captura patrones de comportamiento compartidos,
    pero presenta limitaciones en escenarios de escasez de interacciones.
  </li>
  <li>
    El sistema basado en contenido es independiente del histórico de usuarios,
    pero depende fuertemente de la calidad y riqueza del texto disponible.
  </li>
  <li>
    El sistema híbrido combina ambas fuentes de información, ofreciendo recomendaciones
    más equilibradas y robustas frente a los problemas de dispersión y <i>cold-start</i>.
  </li>
</ul>

<p>
Estos resultados confirman el valor añadido del enfoque híbrido frente a soluciones
basadas en un único paradigma.
</p>

<h3>7.3 Limitaciones del trabajo</h3>

<p>
A pesar de los resultados obtenidos, este trabajo presenta algunas limitaciones que deben
tenerse en cuenta:
</p>

<ul>
  <li>
    No se dispone de datos de interacción explícitos para evaluar la satisfacción real
    de los usuarios finales.
  </li>
  <li>
    La evaluación se basa en métricas indirectas y análisis cualitativos,
    al no contar con un <i>ground truth</i> etiquetado.
  </li>
  <li>
    La calidad del sistema híbrido depende directamente de la disponibilidad de texto
    y de interacciones mínimas entre usuarios y productos.
  </li>
</ul>

<p>
Estas limitaciones son coherentes con el alcance del trabajo y con el contexto académico
del TFM.
</p>

<h3>7.4 Líneas futuras de mejora</h3>

<p>
Como posibles extensiones y mejoras futuras del sistema propuesto, se plantean las siguientes:
</p>

<ul>
  <li>
    Incorporar modelos semánticos más avanzados, como <i>embeddings</i> basados en redes neuronales
    (Word2Vec, Sentence-BERT o similares).
  </li>
  <li>
    Explorar estrategias híbridas más complejas, como combinaciones no lineales
    o modelos de aprendizaje automático que ajusten dinámicamente los pesos.
  </li>
  <li>
    Integrar información adicional, como categorías jerárquicas, marcas o atributos estructurados.
  </li>
  <li>
    Evaluar el sistema con datos reales de interacción de usuarios para aplicar métricas clásicas
    de ranking y satisfacción.
  </li>
</ul>

<p>
Estas líneas permitirían aumentar la calidad de las recomendaciones y aproximar el sistema
a un entorno de producción real.
</p>

<h3>7.5 Conclusión final del trabajo</h3>

<p>
En este Trabajo Final de Máster se han desarrollado tres sistemas de recomendación
(comparables y complementarios), culminando en un sistema híbrido que integra
filtrado colaborativo y recomendación basada en contenido.
</p>

<p>
El trabajo demuestra que, incluso en contextos de datos dispersos y sin feedback explícito,
es posible diseñar sistemas de recomendación coherentes, interpretables y técnicamente sólidos.
</p>

<p>
Los resultados obtenidos validan la aproximación adoptada y sientan las bases para
futuras extensiones más avanzadas en el ámbito de los sistemas de recomendación.
</p>